## Lyrics Fetch

In [48]:
artist_name = 'Ed Sheeran'.replace(' ', '%20')
song = 'Perfect'.replace(' ', '%20')

In [52]:
!curl --request GET -o lyrics_raw.json "https://api.lyrics.ovh/v1/$artist_name/$song"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1566  100  1566    0     0   3235      0 --:--:-- --:--:-- --:--:--  3228


In [55]:
!jq '.lyrics' < lyrics_raw.json > lyrics.json

In [ ]:
lyrics = open("lyrics.json").read()

In [78]:
import re
true_lyric = re.sub('\s+', ' ', lyrics.replace('\\r','').replace('\\\'','\'').replace('\\n', ' ').replace('\n','').replace('"',''))

In [91]:
import boto3
import s3fs
#!pip install tscribe
import tscribe
import json
import urllib.request

In [26]:
s3 = boto3.client('s3')
filename = 'Perfect_normal.mp3'
bucket_name = 'notebook-dylan'
s3.upload_file(filename, bucket_name, filename)

In [27]:
fs = s3fs.S3FileSystem()
file = fs.open('s3://{}/{}'.format(bucket_name, filename))

In [28]:
transcribe = boto3.client('transcribe')

In [36]:
transcribe.start_transcription_job(
    TranscriptionJobName= "Perfect_normal",
    Media={'MediaFileUri': 's3://{}/{}'.format(bucket_name, filename)},
    MediaFormat='mp3',
    LanguageCode='en-US'
)

ConflictException: An error occurred (ConflictException) when calling the StartTranscriptionJob operation: The requested job name already exists. Use a different job name.

In [40]:
status = transcribe.get_transcription_job(TranscriptionJobName='Perfect_normal')
url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
url


'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/996328619957/Perfect_normal/8613971f-9d32-4b15-a96b-130dc028b3c7/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEMn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIDvmEAngg41%2BKA%2FtBiXmRb6myj7leHtRai1XFf1mMz8MAiEAvmuDB7K5Z0gJ7f4BfhPVjByNNZ6D7R39OtVrQdW%2BgW8qtAMIMhACGgwyNzY2NTY0MzMxNTMiDOiHHI5ek3BZhWyrwSqRAwmOw%2BuwuphR%2FBB6oMh10dN8uOOs%2BulWInzfqPm0q0DSHszhBJbsdKdXLsTRzABMYA73pQW1M0WmaUkBPYi%2FTEV8VeRE32Nj8keyWSTu%2FuFtSrcI7x%2FVwLzg2GtyE4USGxTn%2FHlbKn8macwRXMCZjQv8AY2kF4%2FSsOniki0baj619LDHcHm6bTWlZfqnHTp8yKE3Hf8IKQLMunZUMVbHZgIKbh0oJ7ThCBKaoGzeqSvHA9785x%2BNl%2B2Zw8UG1fTKGjZ4Sl3aT9set%2F5nKCrvsOXfDju1%2B7A%2F4Kar6tZZhd93K27QFsdxNuTonTCG8lODiNWODtN8azgMEJRJUZt39rrHDRa%2BCJX7aLvSc9hzhRcSD3%2FnmNimZq4lZJJj3XhAPFSIa8ZZCfBTvPp%2B51wps6UQZa8HmtbEojdLtD6WJN2sm4fSlPDC6gsQfFQ3s1excaruTrIOyHkiED%2ByabAoUgXhYqN8vetDlhjzarJ%2Fqk9r26edqI%2F7qgNK%2FPZQfv9wD4VYOrMVb92egefRiUr4tsXLMKX35oMGOusBMVVdnThFiewssacFcThWuP3O4jn

In [41]:
data = urllib.request.urlopen(url).read().decode()

# parse json object
obj = json.loads(data)
trans_lyric = obj['results']['transcripts'][0]['transcript']
print(trans_lyric)

Uh huh. I found a love for me. Well darling just dive right in. Follow my the well I found a girl beautiful and sweet. Well I never knew you were. There's someone waiting for me because we were just kids when we fell in love not knowing what it was. I will not give you this time darling just kiss me slow your heart iso lyova And in your eyes you're holdin my time baby. I dancing in the dark. Would you between my arms barefoot on the grass listening to our favorite song when you said you looked a mess. I whispered underneath my breath. But you heard it darling, You look perfect night. No. Mm hmm. Well, I found a woman stronger than anyone. I know she shares my dreams. I hope that someday I'll share her home. I found the love to carry more than just my secret to carry large carry Children of home. We are still kiss me with So in love fighting against. I know be alright my darling, Hold my B. My B. I see my future. You're a baby named The Dark with you. Between listening to our favorite s

In [11]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [12]:
#true_lyric = "Wise men say Only fools rush in But I can't help falling in love with you Shall I stay? Would it be a sin If I can't help falling in love with you? Like a river flows Surely to the sea Darling, so it goes Some things are meant to be Take my hand Take my whole life too For I can't help falling in love with you Like a river flows Surely to the sea Darling, so it goes Some things are meant to be Take my hand Take my whole life too For I can't help falling in love with you For I can't help falling in love with you"

In [79]:
true_lyric = true_lyric.replace('.', '').replace(',', '').replace('?', '').replace('!', '').lower()
trans_lyric = trans_lyric.replace('.', '').replace(',', '').replace('?', '').replace('!', '').lower()

In [81]:
true_lyric

"i found a love for me darling just dive right in follow my lead well i found a girl beautiful and sweet oh i never knew you were the someone waiting for me 'cause we were just kids when we fell in love not knowing what it was i will not give you up this time but darling just kiss me slow your heart is all i own and in your eyes you're holding mine baby i'm dancing in the dark with you between my arms barefoot on the grass listening to our favorite song when you said you looked a mess i whispered underneath my breath but you heard it darling you look perfect tonight well i found a woman stronger than anyone i know she shares my dreams i hope that someday i'll share her home i found a love to carry more than just my secrets to carry love to carry children of our own we are still kids but we're so in love fighting against all odds i know we'll be alright this time darling just hold my hand be my girl i'll be your man i see my future in your eyes baby i'm dancing in the dark with you betw

In [82]:
trans_lyric

"uh huh i found a love for me well darling just dive right in follow my the well i found a girl beautiful and sweet well i never knew you were there's someone waiting for me because we were just kids when we fell in love not knowing what it was i will not give you this time darling just kiss me slow your heart iso lyova and in your eyes you're holdin my time baby i dancing in the dark would you between my arms barefoot on the grass listening to our favorite song when you said you looked a mess i whispered underneath my breath but you heard it darling you look perfect night no mm hmm well i found a woman stronger than anyone i know she shares my dreams i hope that someday i'll share her home i found the love to carry more than just my secret to carry large carry children of home we are still kiss me with so in love fighting against i know be alright my darling hold my b my b i see my future you're a baby named the dark with you between listening to our favorite song when i saw you in th

In [80]:
Distance = levenshtein_ratio_and_distance(trans_lyric,true_lyric)
print(Distance)

Ratio = levenshtein_ratio_and_distance(trans_lyric,true_lyric,ratio_calc = True) # do not count upper/lower case as error
print(Ratio)

The strings are 316 edits away
0.8675344563552833


In [87]:
#!pip install audio_metadata
import audio_metadata

metadata = audio_metadata.load('Perfect_normal.mp3')
metadata

<MP3({
    'filepath': '/home/ec2-user/SageMaker/qtm350/CourseAssets/Rekognition_notebook/Perfect_normal.mp3',
    'filesize': '4.08 MiB',
    'pictures': [
        <ID3v2Picture({
            'data': '92.66 KiB',
            'description': '',
            'height': 640,
            'mime_type': 'image/jpeg',
            'type': <ID3PictureType.COVER_FRONT>,
            'width': 640,
        })>,
    ],
    'streaminfo': <MP3StreamInfo({
        'bitrate': '128 Kbps',
        'bitrate_mode': <MP3BitrateMode.CBR>,
        'channel_mode': <MP3ChannelMode.STEREO>,
        'channels': 2,
        'duration': '04:21',
        'layer': 3,
        'protected': False,
        'sample_rate': '44.1 KHz',
        'version': 1,
    })>,
    'tags': <ID3v2Frames({
        'album': ['Perfect (Acoustic)'],
        'artist': ['Ed Sheeran'],
        'comment': [
            <ID3v2Comment({
                'description': 'L64FU3W4YxX3ZFTmbZ+8/RYmUHnK0Z7fNJpXSKoDHIlyHUxeWY7G2db/bSd/55Jv23UKONi+/usMjKHtTxx

In [89]:
metadata['streaminfo']['bitrate']

128000

In [90]:
metadata['streaminfo']['sample_rate']

44100